# Scrap the movies' description using BeautifulSoup

The idea of this notebook is to scrap a website for the description of a movie and use the description to eventually find some "relatedness" between movies

In [1]:
import pandas as pd
import gc
import glob
import numpy as np

In [47]:
# load the file links / we will use the tmdb website and use the ids in the tmdbId column. 
file_path = '/jet/prs/workspace/Movie/ml20m/links.csv'
links_orig = pd.read_csv(file_path)
links_orig.shape, links_orig.columns

((27278, 3), Index(['movieId', 'imdbId', 'tmdbId'], dtype='object'))

In [48]:
links_orig.head(10)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
5,6,113277,949.0
6,7,114319,11860.0
7,8,112302,45325.0
8,9,114576,9091.0
9,10,113189,710.0


In [49]:
links_orig = links_orig[links_orig.tmdbId.notnull()]
links_orig.shape

(27026, 3)

In [120]:
# test some code to download the description of a movie

from bs4 import BeautifulSoup
import requests


start = time.time()

address_intt = 'https://www.themoviedb.org/movie/' + str(45325)
r = requests.get(address_intt)

soup = BeautifulSoup(r.text, "html5lib")
items = soup.find_all('div', {'class' : 'header_info'})

end = time.time()
print(end - start)

items[0].p

0.22960543632507324


<p>A mischievous young boy, Tom Sawyer, witnesses a murder by the deadly Injun Joe. Tom becomes friends with Huckleberry Finn, a boy with no future and no family. Tom has to choose between honoring a friendship or honoring an oath because the town alcoholic is accused of the murder. Tom and Huck go through several adventures trying to retrieve evidence.</p>

In [123]:
0.229*5000

1145.0

In [51]:
def scrap(tmdbId_id):
    try:
        address_intt = 'https://www.themoviedb.org/movie/' + str(int(tmdbId_id))
        r = requests.get(address_intt)

        soup = BeautifulSoup(r.text, "html5lib")
        items = soup.find_all('div', {'class' : 'header_info'})
        return items[0].p
    except:
        return ""

In [63]:
tmdblist = [int(x) for x in links_orig.tmdbId.tolist()]

In [86]:
# we will try to parallelise the fonction on several cpu in an attempt to reduce the time spent on the task:

from multiprocessing.dummy import Pool
from itertools import *
import timeit
import random

import time

start = time.time()

p = Pool(14)
a = p.map(scrap, tmdblist[:5000]) # first 5000 movies
p.close()
p.join()

end = time.time()
print(end - start)

683.5277168750763


In [93]:
links_desc = links_orig.iloc[:5000]
links_desc.shape

(5000, 3)

In [104]:
links_desc['desc']= a

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [107]:
links_desc.desc[0]

<p>Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.</p>

In [115]:
str(toto).replace('<p>','').replace('</p>','')

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [117]:
links_desc.desc = links_desc.desc.apply(lambda x: str(x).replace('<p>','').replace('</p>',''))

/jet/var/python/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [118]:
links_desc.head(10)

,movieId,imdbId,tmdbId,desc
0,1,114709,862.0,"Led by Woody, Andy's toys live happily in his ..."
1,2,113497,8844.0,When siblings Judy and Peter discover an encha...
2,3,113228,15602.0,A family wedding reignites the ancient feud be...
3,4,114885,31357.0,"Cheated on, mistreated and stepped on, the wom..."
4,5,113041,11862.0,Just when George Banks has recovered from his ...
5,6,113277,949.0,"Obsessive master thief, Neil McCauley leads a ..."
6,7,114319,11860.0,An ugly duckling having undergone a remarkable...
7,8,112302,45325.0,"A mischievous young boy, Tom Sawyer, witnesses..."
8,9,114576,9091.0,When a man's daughter is suddenly taken during...
9,10,113189,710.0,James Bond must unmask the mysterious head of ...


In [119]:
links_desc.to_csv('/jet/prs/workspace/Movie/ml20m/movie_desc.csv')